In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install nlpaug
!pip install googletrans
!pip install sacremoses

In [ ]:
import re
import nltk
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Initialize augmenters
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word as naw
from googletrans import Translator
import pandas as pd
import nlpaug.flow as naf
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Text Summarization/LogDataset.csv")
dataset.head()

In [ ]:
dataset["Task Description"][0]

In [ ]:
# shuffled_dataframe = dataset.sample(frac=1).reset_index(drop=True)
shuffled_dataframe = dataset.copy()
# shuffled_dataframe.rename(columns={'Task Description': 'dialogue', 'Summary': 'summary'}, inplace=True)

In [ ]:
def clean_and_lemmatize(text):
    # Remove digits, symbols, punctuation marks, and newline characters
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text.replace('\n', ''))
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Lemmatize each token and join back into a sentence
    lemmatized_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_text
# Apply cleaning and lemmatization to the 'Task Description' column
shuffled_dataframe['Task Description'] = shuffled_dataframe['Task Description'].apply(clean_and_lemmatize)
shuffled_dataframe['Summary'] = shuffled_dataframe['Summary'].apply(clean_and_lemmatize)

In [ ]:
shuffled_dataframe.head()

In [ ]:
shuffled_dataframe["Task Description"][0]

In [ ]:
# Initialize augmenters
aug_synonym = naw.SynonymAug(aug_src='wordnet', aug_p = 0.4)
aug_insert = naw.RandomWordAug(aug_p = 0.3)
# Function to perform all augmentations
def perform_augmentations(text):
    # Synonym Augmentation
    text = aug_synonym.augment(text)

    # Random Word Insertion
    text = aug_insert.augment(text)

    return text

In [ ]:
augmented_data = []
for index, row in shuffled_dataframe.iterrows():
    for _ in range(5):
        augmented_desc = perform_augmentations(row['Task Description'])
        augmented_summ = perform_augmentations(row['Summary'])
        augmented_data.append({'Task Description': augmented_desc, 'Summary': augmented_summ})

In [ ]:
# Creating a DataFrame with augmented text
augmented_df = pd.DataFrame(augmented_data)
# Save augmented data to a CSV file
augmented_df.to_csv('augmented_Dataset_200_data.csv', index=False)

In [ ]:
pd.set_option('display.max_colwidth', -1)
aug_dataset = pd.read_csv("/content/augmented_Dataset_200_data.csv")
aug_dataset.head()

In [ ]:
aug_dataset["Task Description"][0]

In [ ]:
aug_dataset.shape

In [ ]:
import os
dataframes = []
data_file = ["/content/drive/MyDrive/Text Summarization/LogDataset.csv", "/content/augmented_Dataset_200_data.csv"]
for file in data_file:
  # Read the CSV file into a DataFrame
  data = pd.read_csv(file)

  # Append the data to the dataframes list
  dataframes.append(data)

# Concatenate all DataFrames in the list into a single DataFrame
concatenated_data = pd.concat(dataframes, ignore_index=True)

# Save the concatenated data to a new CSV file
concatenated_data.to_csv('Dataset.csv', index=False)


In [ ]:
data = pd.read_csv("/content/Dataset.csv")
data.head()

In [ ]:
data.shape

In [ ]:
data.duplicated().sum()